## DALN Test App

# A. Node Owner

### 1. Create a new session as owner

In [14]:
from weaveapi.records import *
from weaveapi.options import *
from weaveapi.filter import *
from weaveapi.weaveh import *

import pandas as pd

from web3.auto import w3
from eth_account.messages import encode_defunct
from hexbytes import HexBytes

WEAVE_CONFIG = "config/demo_client_remote_daln.config"
nodeApiOwner, sessionOwner = connect_weave_api(WEAVE_CONFIG)

organization = "daln"

data_collection = "vault"
sum_table = "summaries"

{"res":"ok","data":"pong 1694104481656"}


### 2. Create a table to store user data (WARN: removes all previously stored data)

In [15]:
layout = { 
    "columns": { 
        "id": { "type": "LONG", "isIndexed": True, "isUnique": True, "isNullable": False },
        "ts": { "type": "LONG" },
        "writer": { "type": "STRING" },
        "wallet": { "type": "STRING" },
        "sig": { "type": "STRING" },
        "access": { "type": "STRING" },
        "content": { "type": "STRING" },
        "metadata": { "type": "STRING" }
    }, 
    "idColumnIndex": 0,  # Autogenerates IDs
    "timestampColumnIndex": 1, # Fills the column automatically with the network time
    "ownerColumnIndex": 2, # Fills the column automatically with the public key of the writer
    "walletColumnIndex": 3, # Fills the column automatically with the wallet of the writer (if any)
    "signatureColumnIndex": 4, # Fills the column with an EdDSA signature of the record hash
    "allowedRolesColumnIndex": 5, # Fills the column with the allowed reader
    "isLocal": False,
    "applyReadTransformations": True
}

nodeApiOwner.dropTable(sessionOwner, data_collection, sum_table).get()
reply = nodeApiOwner.createTable(sessionOwner, data_collection, sum_table, CreateOptions(False, False, layout)).get()
print(reply)

{'res': 'ok', 'target': {'operationType': 'CREATE_TABLE', 'organization': 'daln', 'account': 'daln', 'scope': 'vault', 'table': 'summaries'}}


# B. User

### 3. Create a new session

- we prove wallet ownership here in order to link our custodial wallet to a blockchain address

In [16]:
from weaveapi.records import *
from weaveapi.options import *
from weaveapi.filter import *
from weaveapi.weaveh import *

from web3.auto import w3
from eth_account.messages import encode_defunct
from hexbytes import HexBytes

keys = nodeApiOwner.generateKeys()
pub = keys[0]
pvk = keys[1]

node_url = "http://ec2-16-171-161-12.eu-north-1.compute.amazonaws.com:443/099cdbbe4213bff84ec9c051749e090e"

chain = "polygon"
wallet = "0x3389798389CD7C54C02dBB253c9f7e6851672Ab5" # Wallet
with open("config/" + wallet[0:6] + ".pvk") as f:     # Expecting a .pvk file with the same name in the jupyter server
    eth_pvk = f.read()

msg = "Please sign this message to confirm you own this wallet\nThere will be no blockchain transaction or any gas fees.\n\nWallet: " + wallet + "\nKey: " + pub
message = encode_defunct(text=msg)
signature = w3.eth.account.sign_message(message, private_key=eth_pvk)
credentials = {
    "account": chain + ":" + wallet,
    "sig": signature.signature.hex(),
    "template": "*",
    "role": "*"
}
print(credentials)
print(pub)
config = weave_client_config(pub, pvk, node_url, organization)
nodeApiUser, sessionUser = connect_weave_api(config, credentials)

{'account': 'polygon:0x3389798389CD7C54C02dBB253c9f7e6851672Ab5', 'sig': '0x9774f6a2e01af64ec2477b5ce8336a487d03c11594c455e5bca06c7851d5d39230f62d9e6ed9054e37f1624c498c83a58c93d96ce227a3a6f1f60a673290bc6a1b', 'template': '*', 'role': '*'}
weavebwBcqSsZYhNLAXcTq7tVxubb7NZa131ho89qvmPS5av7
{"res":"ok","data":"pong 1694104484628"}


### 4. Write a record

In [17]:
rows = [
    [ 
        None, # id, autofilled
        None, # timestamp, autofilled
        None, # writer, autofilled
        None, # wallet, autofilled
        None, # signature, autofilled
        "writer",
        '{"some": "content"}',
        '{"some": "metadata"}'
    ]
]

records = Records(sum_table, rows)
res = nodeApiUser.write(sessionUser, data_collection, records, WRITE_DEFAULT).get()
print(res)

{'res': 'ok', 'target': {'operationType': 'WRITE', 'organization': 'daln', 'account': 'weavebwBcqSsZYhNLAXcTq7tVxubb7NZa131ho89qvmPS5av7', 'scope': 'vault', 'table': 'summaries'}, 'data': 'weaveeGhKYJCZbmB96Qi8MFFqUQFx9u2NFdwK29RcTLZHpp4d,IgXmaH7Z7xUh7KLk8H9Jjtr/T2q/uvaADnadggKLu1g=,4DDy87gHoXmF5ejZzqpZF9LQmPT8tPGQQrxAz9YCSbDvE5dsk5SyGwXBqPm5grKtPHBx7Kpoft63LGEXHBWwChWQ', 'ids': '1'}


**Try to read records**

In [18]:
filter = Filter(None, { "id": "ASC" }, None, None, None)
reply = nodeApiUser.read(sessionUser, data_collection, sum_table, filter, READ_DEFAULT_NO_CHAIN).get()
print(reply)

{'res': 'err', 'target': {'operationType': 'READ', 'organization': 'daln', 'account': 'weavebwBcqSsZYhNLAXcTq7tVxubb7NZa131ho89qvmPS5av7', 'scope': 'vault', 'table': 'summaries'}, 'message': 'Access denied'}


# C. Node Owner

### 5. Read data as owner

In [19]:
filter = Filter(None, { "id": "ASC" }, None, None, None)
reply = nodeApiOwner.read(sessionOwner, data_collection, sum_table, filter, READ_DEFAULT_NO_CHAIN).get()
#print(reply)
df = pd.DataFrame(reply["data"])

display(df.tail())

display(json.loads(df["sig"][0]))

,id,ts,writer,wallet,sig,access,content,metadata
0,1,1694104486667,weavebwBcqSsZYhNLAXcTq7tVxubb7NZa131ho89qvmPS5av7,0x3389798389CD7C54C02dBB253c9f7e6851672Ab5,"[{""sig"":{""recordsHash"":""IgXmaH7Z7xUh7KLk8H9Jjt...",writer,"{""some"": ""content""}","{""some"": ""metadata""}"


[{'sig': {'recordsHash': 'IgXmaH7Z7xUh7KLk8H9Jjtr/T2q/uvaADnadggKLu1g=',
   'count': '1',
   'pubKey': 'weavebwBcqSsZYhNLAXcTq7tVxubb7NZa131ho89qvmPS5av7',
   'sig': '3PF6N88FNJf4Gjd8oLkwE1QvQDLnDojGBABapLD8pdcBkHE8u4BYEYQzYsVrHKGw2bZeaK91U3iweJ2bZzzfKK2p'}}]

### 6. Check merkle tree built from records data

In [20]:
from binarytree import Node

def showtree(tree):
    prev = None
    root = None
    lvl = 1
    for l in tree.split(";"):
        level = l.split(",")
        print(lvl, level)
        lvl += 1
        nodes = []
        for i in range(len(level)):
            pidx = int(i / 2)
            node = Node(level[i][:3] + ".." + level[i][-3:])
            if root is None:
                root = node
            nodes.append(node)
            if prev is not None:
                parent = prev[pidx]
                if parent.left is None:
                    parent.left = node
                else:
                    parent.right = node
        prev = nodes
    
    print(root)

salt = "salt1234" # Same salt used for records hashes, this can be improved to have different salts for *each distinct writer*
digest = "SHA-256"

filter = Filter(None, None, None, None, [ "content", "metadata" ])
reply = nodeApiOwner.merkleTree(sessionOwner, data_collection, sum_table, filter, salt, digest, READ_DEFAULT_NO_CHAIN).get()
tree = reply["data"]["tree"]
rootHash = reply["data"]["rootHash"]
ts = reply["data"]["timestamp"]
rootHashSignature = reply["data"]["signature"]

print("Generated at", ts)
print("Root Hash", rootHash)
print("Signature", rootHashSignature)
print("")
showtree(tree)
# We've built a Merkle Tree at a specific time, signed by the node that created it.
# The Merkle Leaves are salted hashes of the data in the table. Binance did an enumeration of coins.

Generated at 1694104490375
Root Hash FTzxHqefZ9zWHY1cUUD1673NjacYF3TqaJJyAqPWD46R
Signature 2a2dRZjq8P6c6bWH3Dkfwq5xCvEJ8iJcpcmxEbdRLqTbu9Cbv2g7VZLT3gYCHtuqb2TYydZXEP25yMeRZmaRCvN1

1 ['FTzxHqefZ9zWHY1cUUD1673NjacYF3TqaJJyAqPWD46R']

FTz..46R



### 7. Check merkle root hash on chain

https://mumbai.polygonscan.com/address/0xDC3094e824Dc6aaFA42395bfA1e6643f285E5f4B

In [21]:
reply = nodeApiOwner.rootHash(sessionOwner, data_collection, sum_table).get()
data = reply["data"]
display(data)

{'signature': 'NDSbMqQguKPvYLvTAZttZu8QGQhyQRDYouK42oTKj49UPSthVZ2cQUmNegB1Gn2NKGpca97v9GAVVhRvUXPbJDv',
 'rootHash': 'FTzxHqefZ9zWHY1cUUD1673NjacYF3TqaJJyAqPWD46R',
 'timestamp': '1694104502989'}

# D. Anyone who has the data

### 8. Check some record presence in the merkle tree

In [22]:
content = '{"some": "content"}'
metadata = '{"some": "metadata"}'

#Compute a hash of the data. Use the same salt that is agreed with the server
row = [ content, metadata ]
recordHash = nodeApiUser.hashRecord(row, salt)
print(recordHash)

reply = nodeApiUser.verifyMerkleHash(sessionUser, tree, recordHash).get()
print(reply["data"])

FTzxHqefZ9zWHY1cUUD1673NjacYF3TqaJJyAqPWD46R
true


### 9. Validate the root hash against what's stored on chain

In [23]:
reply = nodeApiOwner.rootHash(sessionOwner, data_collection, sum_table).get()
data = reply["data"]
display(data)

chainRootHash = data["rootHash"]
print("\nMatching:", rootHash == chainRootHash) # data hashes are salted. The salt needs to match what's configured on the network to have a match

signingKey = nodeApiUser.sigKey().get()["data"]
print("Node Public Key:", signingKey)

toSign = rootHash + " " + ts
#print(rootHash)
check = nodeApiUser.verifySignature(rootHashSignature, toSign)
print("Check signature from merkleTree() call:", check)

toSign = data["rootHash"] + " " + data["timestamp"]
check = nodeApiUser.verifySignature(data["signature"], toSign)
print("Check signature from blockchain:", check)

# Note, verifySignature isn't unique to Weavechain, it's EdDSA public key cryptography and the verification is done locally

{'signature': 'bB5vCXv4adD5V6Wn1X63w8oyYjezJYKrMHWoEsJw8m9SEPQ8XR6LoNJhwvHmJEEsRJyo46gotBmfVfMt5c6JdCg',
 'rootHash': 'FTzxHqefZ9zWHY1cUUD1673NjacYF3TqaJJyAqPWD46R',
 'timestamp': '1694104507795'}


Matching: True
Node Public Key: GfHq2tTVk9z4eXgyNBGBPNGbgD9crNgwbYnNFfbamJPoxq3cJNYpjRukK3Uf
Check signature from merkleTree() call: True
Check signature from blockchain: True


#### Obtain a merkle proof if needed

In [24]:
reply = nodeApiUser.merkleProof(sessionUser, data_collection, sum_table, recordHash).get()
#print(reply)
data = reply["data"]
display(data)

hashes = [ h.split(",") for h in data["proof"].split(";") ]
display(hashes)

proofSignature = data["signature"]
toSign = data["proof"] + " " + data["timestamp"] # the node signed the proof and the timestamp
check = nodeApiUser.verifySignature(proofSignature, toSign)
print(check)

{'signature': '4KgemWVWGCXV9MpCUAeLLvg3LsbYvgeo8rDzL2W9bbPxLuuajs7dt8DAXEarCP2k8q7yE6LUPBWjZo4zMccgUpEn',
 'rootHash': 'FTzxHqefZ9zWHY1cUUD1673NjacYF3TqaJJyAqPWD46R',
 'proof': 'FTzxHqefZ9zWHY1cUUD1673NjacYF3TqaJJyAqPWD46R',
 'timestamp': '1694104509317'}

[['FTzxHqefZ9zWHY1cUUD1673NjacYF3TqaJJyAqPWD46R']]

True


# E. Node Owner

### 10. Delete a record

In [25]:
nodeApicontent = '{"some": "content"}'
metadata = '{"some": "metadata"}'


# Delete by content, 
#   could also be done by writer: FilterOp.eq("wallet", "0x3389798389CD7C54C02dBB253c9f7e6851672Ab5"),
#   or record id: FilterOp.eq("id", 1),

filter = Filter(
    FilterOp.opand(
        FilterOp.eq("content", content),
        FilterOp.eq("metadata", metadata)
    ), 
    None, None, None, None)
reply = nodeApiOwner.delete(sessionOwner, data_collection, sum_table, filter, DELETE_DEFAULT).get()
print(reply)

{'res': 'ok', 'target': {'operationType': 'DELETE', 'organization': 'daln', 'account': 'daln', 'scope': 'vault', 'table': 'summaries'}, 'data': [{'id': 1, 'ts': '1694104486667', 'writer': 'weavebwBcqSsZYhNLAXcTq7tVxubb7NZa131ho89qvmPS5av7', 'wallet': '0x3389798389CD7C54C02dBB253c9f7e6851672Ab5', 'sig': '[{"sig":{"recordsHash":"IgXmaH7Z7xUh7KLk8H9Jjtr/T2q/uvaADnadggKLu1g=","count":"1","pubKey":"weavebwBcqSsZYhNLAXcTq7tVxubb7NZa131ho89qvmPS5av7","sig":"3PF6N88FNJf4Gjd8oLkwE1QvQDLnDojGBABapLD8pdcBkHE8u4BYEYQzYsVrHKGw2bZeaK91U3iweJ2bZzzfKK2p"}}]', 'access': 'writer', 'content': '{"some": "content"}', 'metadata': '{"some": "metadata"}'}]}


### 11. Verify that it's no longer in the merkle tree

In [26]:
#Compute a hash of the data. Use the same salt that is agreed with the server
row = [ content, metadata ]
recordHash = nodeApiUser.hashRecord(row, salt)
print(recordHash)

filter = Filter(None, None, None, None, [ "content", "metadata" ])
reply = nodeApiOwner.merkleTree(sessionOwner, data_collection, sum_table, filter, salt, digest, READ_DEFAULT_NO_CHAIN).get()

tree = reply["data"]["tree"]
print(tree)

#we use the same record hash that we previously checked to be in the tree
reply = nodeApiUser.verifyMerkleHash(sessionUser, tree, recordHash).get()
print(reply["data"])

FTzxHqefZ9zWHY1cUUD1673NjacYF3TqaJJyAqPWD46R

false
